Import required libraries:

In [ ]:
import requests
import json
import os
import pandas as pd

Create a supplemental function to simplify API 

In [ ]:
def get(url_):
    return json.loads(requests.get(url_).text)

Authenticate to Graph API:

In [ ]:
# TOKEN = "EAACEdEose0cBAEIYEOmJ51kfOHPinwL2ychqVpFT0D9ezMjfe1pMKyH0lP2ZA9RuZBH3wQeJ0EfEboHEqDqy7Gq8PSjIcHgNv1lRyC3ZBMk0CdbdzWckHsqNTRatwPZBvkMBfic2P1jwg5vZAZCHKbHdvFfMjjsHGjeVsY7AqEbpWiUBEb0d1zsKJgJwQw3wTWs6ZBb4WSOOQZDZD"

s = "6cc937f2a9dbc9df92600f365c777d1a"
i = "652869818252649"
u = "https://graph.facebook.com/oauth/access_token?client_id={id}&client_secret={secret}&grant_type=client_credentials"

TOKEN = get(u.format(id=i, secret=s))['access_token']

Setup Graph API host and API node:

In [ ]:
HOST = "https://graph.facebook.com/"
API_NODE = "v2.12/"

Declare custom node - in this case, the 'Technion Confessions' group feed:

In [ ]:
NODE = "{}/feed".format("134517547222780")

Build the requested URL - should be the same for the most of the calls:

In [ ]:
URL = HOST + API_NODE + NODE + "?access_token={}".format(TOKEN)

Create an empty list that will contain the group's posts:

In [ ]:
posts = []

Define a recursive function that will paginate the groups posts and append the new ones to "posts" list:

In [ ]:
def get_posts(url):
    res = get(url)
    if 'paging' in res:
        if 'next' in res['paging']:
            posts.extend(res['data'])
            return get_posts(res['paging']['next'])

If "posts.json" file is not in the directory, the app will get the posts from the API:

In [ ]:
if not "posts.json" in os.listdir(os.getcwd()):
    get_posts(URL)   

If "posts.json" is not in the directory, the app will create it and dump the posts inside it:

In [ ]:
if not "posts.json" in os.listdir(os.getcwd()):
    with open("posts.json", "w") as f:
        f.write(json.dumps(posts))

The app will show the head of the posts table:

In [ ]:
df = pd.read_json(open("posts.json", "r").read())
df.head()

Get the IDs

Get ID of the posts that the post is replying to

Get the posts likes

Get the comments of the posts

Get the likes for each comment

Get the comments for each comment

Get the name of the commenter (for each comment or reply)

Get the commenter gender